## MOST REWARDING PATH PROBLEM (WITH CONSTRAINT ON NEG. EDGES)

### Psuedopolynomial approach in O( |V|.|E|.C)

In [10]:
# Function to create a graph from user input
def create_graph():
    nodes = input("Enter the nodes separated by commas: ").strip().split(',')
    edges = {}
    print("Enter the edges with weights (format: u,v,weight). Type 'done' when finished.")
    while True:
        edge_input = input().strip()
        if edge_input.lower() == 'done':
            break
        u, v, weight = edge_input.split(',')
        edges[(u, v)] = int(weight)
    return nodes, edges

# Function to find the longest path with constraints
def findLongestPathWithConstraints(nodes, edges, start, end, C):
    dp = {node: [-float('inf')] * (C + 1) for node in nodes}
    
    dp[start][C] = 0

    for remainingCost in range(C, -1, -1):
        for u in nodes:
            if dp[u][remainingCost] == -float('inf'):
                continue
            for (u_edge, v_edge), w in edges.items():
                if u_edge == u:
                    if w >= 0:
                        dp[v_edge][remainingCost] = max(dp[v_edge][remainingCost], dp[u][remainingCost] + w)
                    else:
                        absW = abs(w)
                        if remainingCost >= absW:
                            dp[v_edge][remainingCost - absW] = max(dp[v_edge][remainingCost - absW], dp[u][remainingCost] + w)

    longestPath = -float('inf')
    for remainingCost in range(C + 1):
        longestPath = max(longestPath, dp[end][remainingCost])

    return longestPath

# Get graph from user input
nodes, edges = create_graph()
s = input("Enter the source vertex: ")
t = input("Enter the terminal vertex: ")

# Set the constant C
C = int(input("Enter the loss bound constraint: "))

# Find the longest path with the given constraints
longest_path = findLongestPathWithConstraints(nodes, edges, s, t, C)

# Print the result
print(f"The longest path from {s} to {t} with a loss bound of {C} is {longest_path}")


Enter the nodes separated by commas:  s,a,b,c,t


Enter the edges with weights (format: u,v,weight). Type 'done' when finished.


 s,a,1
 a,b,2
 b,c,-4
 c,a,1
 b,t,3
 done
Enter the source vertex:  s
Enter the terminal vertex:  t
Enter the loss bound constraint:  5


The longest path from s to t with a loss bound of 5 is 6


### -----------------------------------------------------------------------------------------------------------------------------------

### Created a separate dict for storing and assigning indices to nodes and added comments

In [ ]:
# Function to create a graph from user input
def create_graph():
    # input all the vertices as : v1,v2,v3,...,vn and create a list for the same
    nodes = input("Enter the nodes separated by commas: ").strip().split(',')
    # create a dict to store the edges in the format {(u,v): weight}
    edges = {}
    print("Enter the edges with weights (format: u,v,weight). Type 'done' when finished.")
    while True:
        edge_input = input().strip()
        if edge_input.lower() == 'done':
            break
        u, v, weight = edge_input.split(',')
        edges[(u, v)] = int(weight)
    return nodes, edges
# Function ends after returning the Vertex and Edge Sets.




# Function to find the longest path with constraints.
# Takes as input parameters: Vertex and Edge Sets, starting and ending vertices & the loss constraint(C).  
# Loss constraint (C) is the absolute value of the maximum allowable sum of negative weights on the path from the source to the terminal vertex.
# Note: c, remainging budget, remainingCost might be used interchangeably henceforth.
def findLongestPathWithConstraints(nodes, edges, start, end, C):
    # V is the size of the vertex set.
    V = len(nodes)
    # create a 2-D dp table with V rows and C+1 columns.
    # dp[v][c] is the most rewarding path to reach vertex v from the source vertex with loss constraint ATMOST C.
    # the rows in the table represent the vertices.
    # the columns in the table represent the value of the remaining budget (starting from C down to 0).
    # Initialize all cells with -inf to start with.
    dp = [[-float('inf')] * (C + 1) for _ in range(V)]
    # node_index is a dict {node : idx} to assign an index value to each vertex instead of having to deal with individual vertex names in the subsequent code fragment.
    # vertex names in general might be difficult to deal with so we assign 0, 1, 2, 3, and so on to the vertices of the Vertex Set.
    node_index = {node: idx for idx, node in enumerate(nodes)}

    # assign dp[source][C] as 0 as the maximum cost to reach the source vertex from itself with the full negative budget is zero.
    dp[node_index[start]][C] = 0

    # loop over the "remaining budget", i.e. from C down to 0; decrementing the remaining budget by 1 on each iteration.
    for remainingCost in range(C, -1, -1):
        # loop over the Vertex set of the graph
        for u in nodes:   # <<<< ################################################################################################################
            # u_idx is the index value assigned to vertex u according to the dict. node_index{node : idx}                                        #
            u_idx = node_index[u]                                                                                                               #
            # if for a vertex u and a remaining cost c, dp[u][c] is found to be -inf; then skip over that vertex entirely and go back to >>>>####
            # the reason being it wont be able to contribute to a path from s to t for a particular remainingCost c, i.e. as there is no way to
            # reach u from the source for a particular remainingCost (represented by -inf); it wont be able to form a path and there is no point
            # in checking what nodes are adjacent to it and any further calculations thereafter.
            if dp[u_idx][remainingCost] == -float('inf'):
                continue
            # loop over all the edges(u_edge,v_edge, weight) in the Edge Set.
            for (u_edge, v_edge), w in edges.items():
                # if u_edge is the vertex u under consideration from the previous loop;
                # then assign v_idx the index value according to the dict. node_index.
                # in other words, we check adjacent vertices v, for the vertex u under consideration from the previous loop. 
                if u_edge == u:
                    v_idx = node_index[v_edge]
                    # if the weight(w), for the edge(u_edge, v_edge) under consideration is positive:
                    # then simply compare which is greater;
                    # (the cell value already assigned to dp[v_edge][c]) OR (the path weight upto u_edge + the edge weight from u_edge to v_edge) 
                    # In other words, check if taking this edge(u_edge, v_edge) results in the path weight(from source to v_edge) being greater than 
                    # the already assigned value(in the table) to reach the vertex v_edge from the source vertex.
                    # Assign to v_idx( i.e. v_edge) the greater of the two values.
                    if w >= 0:
                        dp[v_idx][remainingCost] = max(dp[v_idx][remainingCost], dp[u_idx][remainingCost] + w)
                    # if the weight(w), for the edge(u_edge, v_edge) under consideration is negative:
                    # then first we need to check if we can even consider taking this edge, by checking the current remaining allowable budget.
                    # if we are good to go ahead, then secondly, we need to compare and assign which is greater:
                    # (the current assigned value to the cell dp[v_edge][c-absoluteWeight]) OR (the value of the path till u + taking the negative edge).
                    # Note that the assignment takes place in the column "c-absoluteWeight" and not the column "c" itself unlike the positive weight edges.
                    # This happens as the allowable remaining budget changes on adding a negative edge to the path; which on the other hand ...
                    # would have remained unaffected on adding a positive edge weight.
                    # Also notice the cell dp[v_edge][c-absoluteWeight] might not necessarily always have the value -inf while comapring as ...
                    # the row(or vertex) dp[v_edge] might have been computed before the row(or vertex) dp[u] curently under consideration.
                    else:
                        absoluteWeight = abs(w)
                        if remainingCost >= absoluteWeight:
                            dp[v_idx][remainingCost - absoluteWeight] = max(dp[v_idx][remainingCost - absoluteWeight], dp[u_idx][remainingCost] + w)
    # find the longest path from the row dp[terminal] of the table, since we are interested to know...
    # what is the most rewarding path from the source to the terminal vertex.
    # It is necessary to check every element from the row as the most rewarding path might not necessarily need to use the full negative weight budget. 
    # The path might also be infeasible accorrding to our constraint, in which case we return -inf as the most rewarding path.
    longestPath = -float('inf')
    end_idx = node_index[end]
    for remainingCost in range(C + 1):
        longestPath = max(longestPath, dp[end_idx][remainingCost])

    return longestPath
# Function ends after returning the longest path




# Get graph from user input
nodes, edges = create_graph()
s = input("Enter the source vertex: ")
t = input("Enter the terminal vertex: ")

# Set the constant C, i.e. the maximum allowable negative sum of weights on the path from the source vertex to the terminal vertex.
C = int(input("Enter the loss bound constraint: "))

# Find the most rewarding path from s to t with the given constraints.
longest_path = findLongestPathWithConstraints(nodes, edges, s, t, C)

# Print the result
print(f"The longest path from {s} to {t} with a loss bound of {C} is {longest_path}")
